In [1]:
import xml.etree.ElementTree as ET
import datetime

In [2]:
# Parse the XML file
tree = ET.parse(r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\public\in-the-zone-2\InTheZone2_1.zwo')
# Get the root element
root = tree.getroot()

In [3]:
# Iterate over elements
for child in root:
    print(child.tag, child.attrib)
    for sub_child in child:
        print("\t", sub_child.tag, sub_child.attrib)
        # Access text content of elements if necessary
        print("\t", sub_child.text)

author {}
name {}
description {}
sportType {}
tags {}
	 tag {'name': 'ZONE2'}
	 None
workout {}
	 SteadyState {'Duration': '150', 'Power': '0.5', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.60000002', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.64999998', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.81', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.94999999', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '120.00001', 'Power': '0.5', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '240.00002', 'Power': '0.80000001', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '120', 'Power': '0.5', 'pace': '0'}
	 
            
	 Ramp {'Duration': '360', 'PowerLow': '0.51930803', 'PowerHigh': '0.69999993', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '180.00002', 'Power': '0.69999999', 'pace': '0'}
	 
            
	 Ramp

In [5]:
def parse_workout_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    workout_data = []
    intervals = []
    tagsToFind = ['SteadyState','Ramp','IntervalsT','Cooldown']
    for element in root.find(".//workout"):
        if element.tag in tagsToFind:
            intervals.append(element)
    
    for interval in intervals:
        type = interval.tag
        duration = interval.attrib.get("Duration")
        power = interval.attrib.get("Power")
        power_low = interval.attrib.get("PowerLow")
        power_high = interval.attrib.get("PowerHigh")
        onPower = interval.get('OnPower', None)
        offPower = interval.get('OffPower', None)
        onDuration = interval.get('OnDuration', None)
        offDuration = interval.get('OffDuration', None)
        repeat = interval.get('Repeat', None)

        # Convert attributes to float if they exist
        if duration is not None:
            duration = float(duration)
        if power is not None:
            power = float(power)
        if power_low is not None:
            power_low = float(power_low)
        if power_high is not None:
            power_high = float(power_high)
        if onPower is not None:
            onPower = float(onPower)
        if offPower is not None:
            offPower = float(offPower)
        if onDuration is not None:
            onDuration = float(onDuration)
        if offDuration is not None:
            offDuration = float(offDuration)
        if repeat is not None:
            repeat = int(repeat)
            
        interval_data = {
            "type": type,
            "duration": duration,
            "power": power,
            "powerLow": power_low,
            "powerHigh": power_high,
            "onPower": onPower,
            "offPower": offPower,
            "onDuration": onDuration,
            "offDuration": offDuration,
            "repeat": repeat
        }
        workout_data.append(interval_data)

    return workout_data


In [6]:
def color(power):
    if power <= 59:
        color = 'grey'
    elif power<=75:
        color = 'blue'
    elif power<=89:
        color = 'green'
    elif power<=104:
        color = 'yellow'
    elif power<=118:
        color = 'orange'   
    else:
        color = 'red'
    return color   

In [117]:
def split_interval_into_buckets(interval, buckets = [[0, 59], [60, 75], [76, 89], [90, 104], [105, 118],[119,1000]]):
  start, end = interval
  counts = [0] * len(buckets)  # Initialize counts for each bucket

  # Loop through each bucket
  for i, bucket in enumerate(buckets):
    bucket_start, bucket_end = bucket
    # Check if the interval overlaps with the current bucket
    if start <= bucket_end and end >= bucket_start:  # Adjusted condition
      # Calculate the overlap within the bucket
      overlap_start = max(start, bucket_start)
      overlap_end = min(end, bucket_end)
      # Increment the count for the current bucket based on the overlap
      counts[i] += overlap_end - overlap_start + 1

  return counts

# Example usage
interval = [65, 55]
buckets = [[0, 59], [60, 75], [76, 89], [90, 104], [105, 118],[119,1000]]
counts = split_interval_into_buckets(sorted(interval), buckets)

print(counts)  # Output: [16, 10, 15] (assuming 0-based indexing)


[5, 6, 0, 0, 0, 0]


In [131]:
for i in reversed(range(6)):
    print(i)

5
4
3
2
1
0


In [133]:
time = 100
duration = 150
powerLeft = 0.6
powerRight = 0.4
timeScale=0.1

d = datetime.timedelta(seconds=round(duration))
d = '{:2d}:{:02d}'.format(d.seconds // 60, d.seconds % 60)
def pathDef(time, duration, powerLeft_, powerRight_, timeScale, color):
    path_ = f"""<Tooltip label = "{d}min ramp from {round(powerLeft*100)}% to {round(powerRight*100)}%">
    <path d="M {time} 0 L {time} {powerLeft_*100} L {time + duration*timeScale} {powerRight_*100} L {time + duration*timeScale} {0} 0 Z" fill="{color}" stroke="{color}" />
    </Tooltip>"""
    return path_

dur_pow = duration/(abs(powerLeft - powerRight)*100+1)

path = []
powerStart = powerLeft
for i in reversed(range(6)):
    part = split_interval_into_buckets(sorted([powerLeft*100, powerRight*100]))[i]
    if part > 0:
        path.append(pathDef(time, part * dur_pow, powerStart,powerStart-part/100, timeScale, color(buckets[i][1])))
        powerStart = powerStart - part/100
        time = time + part*dur_pow * timeScale
path

['<Tooltip label = " 2:30min ramp from 60% to 40%">\n    <path d="M 100 0 L 100 60.0 L 100.71428571428571 59.0 L 100.71428571428571 0 0 Z" fill="blue" stroke="blue" />\n    </Tooltip>',
 '<Tooltip label = " 2:30min ramp from 60% to 40%">\n    <path d="M 100.71428571428571 0 L 100.71428571428571 59.0 L 115.0 38.99999999999999 L 115.0 0 0 Z" fill="grey" stroke="grey" />\n    </Tooltip>']

In [110]:
type(sorted([powerLeft*100, powerRight*100]))

list

In [91]:
split_interval_into_buckets(sorted([powerLeft*100, powerRight*100]))

[4.999999999999993, 1.0, 0, 0, 0, 0]

['<Tooltip label = " 2:30min ramp from 50% to 100%">\n    <path d="M 115.0 0 L 115.0 50.0 L 117.94117647058823 60.0 L 117.94117647058823 0 0 Z" fill="grey" stroke="grey" />\n    </Tooltip>',
 '<Tooltip label = " 2:30min ramp from 50% to 100%">\n    <path d="M 117.94117647058823 0 L 117.94117647058823 60.0 L 122.6470588235294 76.0 L 122.6470588235294 0 0 Z" fill="blue" stroke="blue" />\n    </Tooltip>',
 '<Tooltip label = " 2:30min ramp from 50% to 100%">\n    <path d="M 122.6470588235294 0 L 122.6470588235294 76.0 L 126.76470588235294 90.0 L 126.76470588235294 0 0 Z" fill="green" stroke="green" />\n    </Tooltip>',
 '<Tooltip label = " 2:30min ramp from 50% to 100%">\n    <path d="M 126.76470588235294 0 L 126.76470588235294 90.0 L 130.0 101.0 L 130.0 0 0 Z" fill="yellow" stroke="yellow" />\n    </Tooltip>']

In [134]:
def box(time, duration, power, timeScale):
    d = datetime.timedelta(seconds=round(duration))
    d = '{:2d}:{:02d}'.format(d.seconds // 60, d.seconds % 60)
       
    path = f"""<Tooltip label = "{d}min @{round(power*100)}%">
    <path d="M {time} 0 L {time} {power*100} L {time + duration*timeScale} {power*100} L {time + duration*timeScale} {0} 0 Z" fill="{color(power*100)}" stroke="{color(power*100)}" />
    </Tooltip>"""
    return path

def trapez(time, duration, powerLeft, powerRight, timeScale):
    d = datetime.timedelta(seconds=round(duration))
    d = '{:2d}:{:02d}'.format(d.seconds // 60, d.seconds % 60)
    def pathDef(time, duration, powerLeft_, powerRight_, timeScale, color):
        path_ = f"""<Tooltip label = "{d}min ramp from {round(powerLeft*100)}% to {round(powerRight*100)}%">
        <path d="M {time} 0 L {time} {powerLeft_*100} L {time + duration*timeScale} {powerRight_*100} L {time + duration*timeScale} {0} 0 Z" fill="{color}" stroke="{color}" />
        </Tooltip>"""
        return path_
    dur_pow = duration/(abs(powerLeft - powerRight)*100+1)
    path = []
    powerStart = powerLeft
    if powerLeft > powerRight:
        for i in reversed(range(6)):
            part = split_interval_into_buckets(sorted([powerLeft*100, powerRight*100]))[i]
            if part > 0:
                path.append(pathDef(time, part * dur_pow, powerStart,powerStart-part/100, timeScale, color(buckets[i][1])))
                powerStart = powerStart - part/100
                time = time + part*dur_pow * timeScale
    else:
        for i in range(6):
            part = split_interval_into_buckets(sorted([powerLeft*100, powerRight*100]))[i]
            if part > 0:
                path.append(pathDef(time, part * dur_pow, powerStart,powerStart+part/100, timeScale, color(buckets[i][1])))
                powerStart = powerStart + part/100
                time = time + part*dur_pow * timeScale
        
    return path
    

In [135]:
def generate_paths(workout_data):
    paths = []
    totalTime = 0
    intervals = 0
    gap = 2
    
    for interval in workout_data:
        type = interval['type']
        duration = interval.get('duration', None)
        onDuration = interval.get('onDuration', None)
        offDuration = interval.get('offDuration', None)
        repeat = interval.get('repeat', None)
                
        if type in ["SteadyState","Ramp","Cooldown"]:
            totalTime = totalTime + duration
            intervals = intervals + 1
        if type == "IntervalsT":
            totalTime = totalTime + repeat*(onDuration + offDuration)
            intervals = intervals + repeat*2
    timeScale = (1000 - intervals * gap)/totalTime
    print(totalTime)
    
    time = 0
    for interval in workout_data:
        type = interval['type']
        duration = interval.get('duration', None)
        power = interval.get('power', None)
        powerLow = interval.get('powerLow', None)
        powerHigh = interval.get('powerHigh', None)
        onPower = interval.get('onPower', None)
        offPower = interval.get('offPower', None)
        onDuration = interval.get('onDuration', None)
        offDuration = interval.get('offDuration', None)
        repeat = interval.get('repeat', None)
        
        
        if type == "SteadyState":
            path = box(time, duration, power, timeScale)
            paths.append(path)
            time = time + duration*timeScale + gap
        if type in ["Ramp","Cooldown"]:
            path = trapez(time, duration, powerLow, powerHigh, timeScale)
            paths.extend(path)
            time = time + duration*timeScale + gap
        if type == "IntervalsT":
            
            for i in range(repeat):
                path = box(time, onDuration, onPower, timeScale)
                paths.append(path)
                time = time + onDuration*timeScale + gap
                path = box(time, offDuration, offPower, timeScale)
                paths.append(path)
                time = time + offDuration*timeScale + gap
    
            
    return paths


In [164]:
root.find('description').text

'Zone 2 training is one of the most important aspects of any training plan. We do not always get faster by training faster.\n\nIn Zone 2 training we are stimulating your type I muscle fibers - slow twitch.\n\nSpending time in Zone 2 is absolutely essential to improving performance. This workout is a mashup of shorter zone 2 workouts (Zone 2 £#1, Cadence Pyramid & Aerobic Threshold Development) to get a long and varied zone 2 workout.'

In [165]:
def generate_svg(paths):
    svg_start = """
    import {Box, Tooltip, Heading, Text} from '@chakra-ui/react'
    import './css/Workout.css'

    export const WorkoutGraphSVG = () => (
    <div>
    <Heading as ='h2' >Description</Heading>
    <Text>"""
    svg_des = root.find('description').text
    svg_middle = """</Text>
    <Box as='svg'
    viewBox='0 0 1000 100'
    width='100%'
    height={'auto'}
    transform= 'scaleY(-1)'>
    <svg>"""
    svg_end = "</svg></Box></div>);"
    svg_content = "\n".join(paths)
    return svg_start + svg_des + svg_middle +  svg_content + svg_end

In [166]:
def save_svg_to_tsx(svg_content, file_path):
    with open(file_path, 'w') as f:
        f.write(svg_content)

In [167]:
# Example usage
workout_file = r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\public\in-the-zone-2\InTheZone2_1.zwo'
workout_data = parse_workout_file(workout_file)
svg_paths = generate_paths(workout_data)
svg_content = generate_svg(svg_paths)

file_path = r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\app\in-the-zone-2\workouts\nextWorkout.tsx'
save_svg_to_tsx(svg_content, file_path)

# Now you can use svg_paths to embed the SVG paths in your SVG container or save them to an SVG file.
# For example, you can join the svg_paths list into a single string and embed it in an SVG element in an HTML document.


7200.000236000003
